# **Part 2: Trainning & Playing to stabilize Cart-Pole Agent.**

- `Name` : **Pavaris Asawakijtananont**
- `Number` : 65340500037

#### **Testing Configuration and Control Variable**

In this part we start to training agent from small space and small action space to ensure that agent can visit all state and update all possible action to learn to stabilize cartpole.
- at first we settup all algorithm with same configuration space consisting with
    - number of action
    - action range
    - number of bins / number of state (for discretize to discrete space)
    - number of episode
    - start epsilon
    - final epsilon
    - epsilon decay rate
    - learning rate
    - discount factor
    - boundary of state

- configure for agent and learning algorithm
```python
    num_of_action = 5
    action_range = [-12, 12]  # [min, max]
    discretize_state_weight = [4, 8, 4, 4]  # [pose_cart:int, pose_pole:int, vel_cart:int, vel_pole:int]
    learning_rate = 0.03
    n_episodes = 5000
    start_epsilon = 1.0
    epsilon_decay = 0.00003 # reduce the exploration over time
    final_epsilon = 0.05
    discount = 1
```
- configure for discretize space
```python
    pose_cart_bound = 3
    pose_pole_bound = float(np.deg2rad(24.0))
    vel_cart_bound = 15
    vel_pole_bound = 15
```

#### **Agent State Space**

for ensure that agent can visit all possible state we set a small state space and calculating to a episode, the agent have

$$
N_S = S_{pose_cart}*S_{pose_pole}*S_{vel_cart}*S_{vel_cart} = 512

N_Q = N_S * N_A = 2560
$$

all space of action value is less than episode that mean it is highly chance to visit most of all state and update action value of pair of state action many time. 

- for visit a state it depend on exploration and exploitation rate is decay epsilon rate that we use linear decay

$$
    1-decay \cdot step = 0.05
$$
$$
    step \approx 3333
$$

the agent will start quite fully exploitation or update optimal action from learning after 3333 step

## **Base-line Experiment**

#### **Reward Setting**
- for first experiment we setting the same reward function, terminate conditions and all configuration in environment to test all algorithm.

- you can see more in `stabilize_cartpole_env_cfg.py`

```python
class RewardsCfg:
    """Reward terms for the MDP."""

    # (1) Constant running reward
    alive = RewTerm(func=mdp.is_alive, weight=1.0)
    # (2) Failure penalty
    terminating = RewTerm(func=mdp.is_terminated, weight=-2.0)
    # (3) Primary task: keep pole upright
    pole_pos = RewTerm(
        func=mdp.joint_pos_target_l2,
        weight=-1.0,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["cart_to_pole"]), "target": 0.0},
    )
```
|Task | Reward| Description |
|---|---|---|
| Alive | 1 | agent aliving reward |
| Terminate | -2 | penalty when terminate |
| Pole Position | -1 | penalty when agent not in target (0)|


    # Clipping value
    pose_cart_bound = 3
    pose_pole_bound = float(np.deg2rad(24.0))
    vel_cart_bound = 15
    vel_pole_bound = 15
    
    num_of_action = 5
    action_range = [-12, 12]  # [min, max]
    discretize_state_weight = [4, 8, 4, 4]  # [pose_cart:int, pose_pole:int, vel_cart:int, vel_pole:int]
    learning_rate = 0.03
    n_episodes = 5000
    start_epsilon = 0.05
    epsilon_decay = 0.00003 # reduce the exploration over time
    final_epsilon = 0.05
    discount = 1

### **Q-Learning**

<div style="text-align: center;">
    <video width="750" controls>
        <source src="video/simple_QL.mp4" type="video/mp4">
    </video>
</div>

- In `Q Learning` Cartpole can learn to stabilize but when time run the agent is moving to boundary of **[-3 ,3]** and agent is terminating, that mean agent is learn to stabilize but agent is still not fully learn to not out of bound, It's can from not enought velocity action to cartpole to swing the pole to opposite direction to change cart position.

### **Double Q-Learning**

<div style="text-align: center;">
    <video width="750" controls>
        <source src="video/simple_DQL.mp4" type="video/mp4">
    </video>
</div>

- agent learn with `Double Q Learning` also can learn to stabilzie too, but someway agent look like doesn't learn to not terminate by out of boundary, and agent look like got limit velocity and it can't swing the pole to another direction that make cart out of bound  

### **SARSA**

<div style="text-align: center;">
    <video width="750" controls>
        <source src="video/simple_SARSA.mp4" type="video/mp4">
    </video>
</div>

- `SARSA` is same as a Double Q Learning but `SARSA` is mostly learn to stabilize in to right direction

### **Monte Carlo**

<div style="text-align: center;">
    <video width="750" controls>
        <source src="video/simple_MC0.mp4" type="video/mp4">
    </video>
</div>

- `Monte Carlo` is can't learn to stabilize with default configuration but it learn to **terminate** instead
- to make agent learn with `Monte Carlo` we need to set some configuration is **`Discount Factor`** for more detail will discussion in **PART 3** 

**Changing Discount Factor**
- I have reduce Discout Factor from 1 to 0.25 and 0.01 

<div style="display: flex; justify-content: center; gap: 20px;">
    <video width="750" controls>
        <source src="video/simple_MC1.mp4" type="video/mp4">
    </video>
    <video width="750" controls>
        <source src="video/simple_MC2.mp4" type="video/mp4">
    </video>
</div>

*Left : Discout Factor = 0.25 , Right : Discount Factor = 0.01*

**Discount Factor : 0.25** - agent can learn to stabilize but to perform stabilize still not good, the pole is swing over time to stabilize

**Discount Factor : 0.01** - agent also learn to stabilize but cart can't change direction to not terminate, behavior of this agent is like a SARSA or Double Q learning but this agent can learn to stabilize with 2 direction

### **Summary**

- Every algorithm can enable the agent to learn to stabilize the pole, but different algorithms perform differently under the same configuration. Each algorithm has its own pros and cons. For example, Monte Carlo is a high-variance method that can fail if we weight future rewards too heavily (or if the final reward at termination dominates). In contrast, methods that use bootstrapping to predict the next action and state from the current policy can introduce high bias. Despite these trade-offs, off-policy methods still manage to stabilize the pole.
- **`Q-Learning`** is the best agent for the stabilization task; however, it also has a problem: the agent does not learn to change direction when the cart is about to go out of bounds, similar to other algorithms. To address this issue, increasing the range of actions to allow higher velocities might help swing the pole in the opposite direction. Currently, the cart is merely following the pole to stabilize it, which causes the cart to move out of bounds.

## **Stabilize with larger space**

- in this experiment we will setting larger space in observation term and action space to improving stabilizing performance to observe performance of agent with difference number of training episode and improve stabilize performance

**Independent Variable**
- number of episode to train

**Dependent Variable**
- Performance in task(stabilize performance)
- Q table (Part 3)

**Control Variable**
- Learning Algorithm : Q Learning
- Environment Configuration
- Reward Function

```python
    # Hyper Parameter
    num_of_action = 11
    action_range = [-25, 25]  # [min, max]
    discretize_state_weight = [7, 12, 5, 5]  # [pose_cart:int, pose_pole:int, vel_cart:int, vel_pole:int]
    learning_rate = 0.03
    n_episodes = 5000
    start_epsilon = 1.0
    epsilon_decay = 0.00003 # reduce the exploration over time
    final_epsilon = 0.05
    discount = 1

    # Clipping value Parameter for discretize observation term
    pose_cart_bound = 3
    pose_pole_bound = float(np.deg2rad(24.0))
    vel_cart_bound = 25
    vel_pole_bound = 25
```

### **Q Learning**


<div style="display: flex; justify-content: center; align-items: center;">
  <video width="750" controls>
    <source src="video/mod_QL_20000.mp4" type="video/mp4">
  </video>
</div>


- When we expand the observation space (i.e., increase the state space available to the agent) and enlarge the action space along with its range, it affects the agent’s performance in stabilizing the pole. With the increased space, the agent learns to swing the cart back to the center to avoid termination. In contrast, with the previous configuration, the agent did not learn to return to the center and episodes frequently terminated.

### **Double Q Learning**

## **Swing Up**